In [1]:
import cv2
import os

def load_images_from_directory(directory):
    image_list = []
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".jpg"):
                img = cv2.imread(os.path.join(root, filename))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                image_list.append(img)
    return image_list

# Load grayscale and color images from directories with subfolders
grayscale_images = load_images_from_directory("bw_output/")
color_images = load_images_from_directory("color_input/")

In [2]:
# Create a simple CNN-based autoencoder model for image colorization.
# Define the encoder and decoder using a deep learning framework like Keras.

import tensorflow as tf
from tensorflow.keras import layers, models

def create_colorization_model():
    model = models.Sequential()

    # Encoder
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 1)))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    
    # Decoder
    model.add(layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    
    # Output layer with 3 channels (RGB)
    model.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'))
    
    return model

In [3]:
# Step 3: Loss Function

# Choose a suitable loss function. Here, we'll use Mean Squared Error (MSE).

def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Compile the model with the loss function
model.compile(optimizer='adam', loss=custom_loss)

NameError: name 'model' is not defined

In [ ]:
# Step 4: Training

# Train the model on grayscale and color images
model.fit(np.array(grayscale_images), np.array(color_images), epochs=10, batch_size=32, validation_split=0.2)
